In [1]:
import pandas as pd
import os
import numpy as np
import soundfile as sf
from IPython.display import clear_output
import pickle
import librosa
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow.keras.models import load_model

In [ ]:
# Wczytuję wszytkie potrzebne narzędia doa obliczenai embeddingu jak i przeprowadzenia postprocessingu

model = load_model("/kaggle/input/cnn_mel/keras/default/1/model.h5");

with open("/kaggle/input/tools-for-cross-validation-mel/scaler.pkl", "rb") as file:
    scaler_before_embedding = pickle.load(file)
    
with open("/kaggle/input/tools-for-cross-validation-mel/scaler_after_embedding.pkl", "rb") as file:
    scaler_after_embedding = pickle.load(file)

with open("/kaggle/input/tools-for-cross-validation-mel/lda.pkl", "rb") as file:
    lda = pickle.load(file)

In [ ]:
def split_audio_to_slices(path_to_files, seconds):
    
    # Przechodzę do katalogów wewnątrz folderu osoby (ID osoby).
    # Każdy folder wewnętrzny zawiera więcej podfolderów, które mogą zawierać nagrania.
    paths_inside_ID = [f.name for f in os.scandir(path_to_files) if f.is_dir()]

    # Tworzę pełne ścieżki do podfolderów, aby przejść do wszystkich plików nagrań dla danej osoby.
    full_paths_to_files = [path_to_files + '/' + path_inside_ID for path_inside_ID in paths_inside_ID]

    # Zbieram wszystkie ścieżki do plików audio danej osoby.
    # Każdy plik powinien mieć rozszerzenie `.flac`, a wszystkie pliki są przechowywane w zmiennej `all_files_for_ID`.
    
    all_files_for_ID = []
    for full_path_to_files in full_paths_to_files:
        files = [f.name for f in os.scandir(full_path_to_files) if f.is_file() and f.name.endswith('.flac')]
        files = [full_path_to_files + '/' + file for file in files]
        all_files_for_ID = all_files_for_ID + files

    # Łączę wszystkie nagrania danej osoby w jedno bardzo długie nagranie.
    # Używam częstotliwości próbkowania 16kHz.
    sr = 16000
    combined_signals = np.array([])
    for file_for_ID in all_files_for_ID:
        signal, sr = librosa.load(file_for_ID, sr=sr)
        combined_signals = np.concatenate([combined_signals, signal])

    # Długie nagranie dzielę na  fragmenty o podanej długości.
    # Fragmenty, które mają mniej niż zadeklarowane długości nagrania (resztki na końcu nagrania), są pomijane.
    list_for_parts = []
    len_of_combined_signals = len(combined_signals)
    step = seconds * sr  # Ustawienie skoku na 5 sekund
    for i in np.arange(start=0, stop=len_of_combined_signals-step, step=step):
        list_for_parts.append(combined_signals[i:i+step].tolist())

    
    parts = np.array(list_for_parts)
    
    
    # Funkcja zwraca pociętę na kawałki, o długości 1 sekundy nagrania
    return parts
    

In [ ]:
class EbeddingExtractor:
    
    def __init__(self, model, scaler_before_embedding, scaler_after_embedding, lda):
        self.model = model  # Przypisanie modelu do obiektu
        self.lda = lda  # Przypisanie modelu LDA do obiektu
        self.scaler_before_embedding = scaler_before_embedding  # Skaler do standaryzacji mel-spektorgramów przed generowaniem embeddingów
        self.scaler_after_embedding = scaler_after_embedding  # Skaler do standaryzacji embeddingów przed zastosowaniem LDA

    # Funkcja obliczająca mel-spektorgramy dla danego nagrania audio
    def calucate_mel(self, audio):
        quantity_of_mel_filters = 64

        # Obliczanie mel-spektorgramów  za pomocą librosa
        mel = librosa.feature.melspectrogram(y=audio, 
                                            sr=16000,  
                                            n_mels=quantity_of_mel_filters).T
        
        # Standaryzacja mel-spektorgramów przed generowaniem embeddingów
        mel = self.scaler_before_embedding.transform(mel)
        return mel

    # Funkcja obliczająca embedding na podstawie wcześniej przetworzonych mel-spektorgramów 
    def calcuate_embedding(self, audio_mel):
        
        intermediate_layer_model = Model(inputs=self.model.inputs,
                                         outputs=self.model.get_layer('bottleneck').output)
        intermediate_output = intermediate_layer_model.predict(audio_mel[np.newaxis, ...])
        
        return intermediate_output

    # Funkcja postprocessingu embeddingu
    def transform_audio_postprocessing(self, embedding):
        embedding = self.scaler_after_embedding.transform(embedding)  # Standaryzacja embeddingu
        embedding = self.lda.transform(embedding)  # Użycie LDA
        
        return embedding

# Tworzenie obiektu klasy EmbeddingExtractor z przekazaniem modelu, skalerów i modelu LDA
EmbExtr = EbeddingExtractor(model=model, 
                            scaler_before_embedding=scaler_before_embedding,
                            scaler_after_embedding=scaler_after_embedding,
                            lda=lda)

In [ ]:
with open("/kaggle/input/data-to-enrollment-and-test/data_to_cross_checking.pkl", "rb") as file:
    data_to_cross_checking = pickle.load(file)

In [ ]:
embedding_all_people = []  # Lista do przechowywania embeddingów dla wszystkich osób
embedding_all_people_postprocessed = []
# Pętla przetwarzająca nagrania dla każdej z 50 osób
# Podana jest liczbaa 10 ponieważ zadanei podzieliłem na 5 kont kaggle
for i in range(0, 10):
    
    # Podział nagrania danej osoby na fragmenty 1-sekundowe za pomocą funkcji split_audio_to_slices
    slices = split_audio_to_slices(data_to_cross_checking[i], seconds=1)
    embedding_one_person = []  # Lista do przechowywania embeddingów dla pojedynczej osoby
    embedding_one_person_postprocessed = []
    
    iterator = 0  # Inicjalizacja iteratora do śledzenia postępu przetwarzania
    for slice in slices:
        # Obliczanie mel-spektorgramów dla jednego fragmentu audio
        mel = EmbExtr.calucate_mel(slice)
        
        # Generowanie embeddingu z obliczonych mel-spektorgramów 
        embedding = EmbExtr.calcuate_embedding(mel)
        
        # Postprocessing embeddingu - LDA
        postprocessed_embedding = EmbExtr.transform_audio_postprocessing(embedding)
        
        # Dodanie przetworzonego embeddingu do listy embeddingów danej osoby
        embedding_one_person_postprocessed.append(postprocessed_embedding)
        embedding_one_person.append(embedding)
        
        # Aktualizacja iteratora i wyświetlanie postępu przetwarzania
        iterator += 1
        print(i + iterator / len(slices))  # Procentowy postęp przetwarzania dla osoby o ID i
        clear_output(wait=True)
    
    # Dodanie listy embeddingów danej osoby do głównej listy embeddingów dla wszystkich osób
    embedding_all_people.append(embedding_one_person)
    embedding_all_people_postprocessed.append(embedding_one_person_postprocessed)

In [ ]:
with open('embedding_all_people.pkl', 'wb') as file:
    pickle.dump(embedding_all_people, file)

with open('embedding_all_people_postprocessed.pkl', 'wb') as file:
    pickle.dump(embedding_all_people_postprocessed, file)